In [1]:
import sqlite3
import pandas as pd

Schema Description

In [3]:
# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables and insert sample data
cursor.executescript("""
CREATE TABLE video_views (
  viewer_id INT,
  video_id INT,
  view_time INT,
  view_date TEXT
);

CREATE TABLE videos (
  video_id INT,
  category TEXT
);

INSERT INTO video_views VALUES
(1, 101, 120, '2024-01-01'),
(2, 101, 150, '2024-01-01'),
(3, 102, 300, '2024-01-01'),
(4, 103, 200, '2024-01-01'),
(5, 104, 50,  '2024-01-01'),
(6, 102, 250, '2024-01-02'),
(7, 104, 80,  '2024-01-02'),
(8, 105, 100, '2024-01-02'),
(9, 102, 200, '2024-01-03'),
(10, 104, 300,'2024-01-03');

INSERT INTO videos VALUES
(101, 'Music'),
(102, 'Education'),
(103, 'Gaming'),
(104, 'Music'),
(105, 'News');
""")


Visualizing Tables

In [4]:
query1 = """
select * from video_views
"""

query2 = """
select * from videos
"""

res1 = pd.read_sql_query(query1, conn)
res1

,viewer_id,video_id,view_time,view_date
0,1,101,120,2024-01-01
1,2,101,150,2024-01-01
2,3,102,300,2024-01-01
3,4,103,200,2024-01-01
4,5,104,50,2024-01-01
5,6,102,250,2024-01-02
6,7,104,80,2024-01-02
7,8,105,100,2024-01-02
8,9,102,200,2024-01-03
9,10,104,300,2024-01-03


In [5]:
res2 = pd.read_sql_query(query2, conn)
res2

,video_id,category
0,101,Music
1,102,Education
2,103,Gaming
3,104,Music
4,105,News


SOLUTION

STEP 1 = We're grouping by category and by date so we get for each category the date and the total view time for that for each day

In [8]:
query_st1 = """
select v.category, vv.view_date, sum(vv.view_time) as Total_View_Time
from video_views vv
join videos v on v.video_id = vv.video_id
group by v.category, vv.view_date
"""

res_st1 = pd.read_sql_query(query_st1, conn)
res_st1

,category,view_date,Total_View_Time
0,Education,2024-01-01,300
1,Education,2024-01-02,250
2,Education,2024-01-03,200
3,Gaming,2024-01-01,200
4,Music,2024-01-01,320
5,Music,2024-01-02,80
6,Music,2024-01-03,300
7,News,2024-01-02,100


STEP 2 = We wrap up the previous query in a CTE so we can use those result in a further comparison of dates ranks <br> 
and <br> 
STEP 3 = From the ranked date with view time amount we calculated, we can now get the highest of every date

In [17]:
query_st2 = """
with category_view_time_per_day as (
select v.category, vv.view_date, sum(vv.view_time) as Total_View_Time
from video_views vv
join videos v on v.video_id = vv.video_id
group by v.category, vv.view_date
), 
ranked_days as (
select *,
row_number() over(
partition by category
order by Total_View_Time desc
) as ranked
from category_view_time_per_day
)
select category, view_date, Total_View_Time
from ranked_days
where ranked=1;
"""

res_st2 = pd.read_sql_query(query_st2, conn)
res_st2

,category,view_date,Total_View_Time
0,Education,2024-01-01,300
1,Gaming,2024-01-01,200
2,Music,2024-01-01,320
3,News,2024-01-02,100
